In [1]:
import os
import pandas as pd
from src.data_utils import clean_text
from transformers import BertTokenizerFast

c:\Users\tatya\OneDrive\Documents\auto-completion-of-texts\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
# Read txt-file
with open('data/raw_dataset.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# delete \n
texts = [line.strip() for line in lines if line.strip()]

# clean data
cleand_dataset = [clean_text(text) for text in texts]


In [ ]:
cleand_dataset[:5]

[' switchfoot http twitpic com 2y1zl awww that s a bummer you shoulda got david carr of third day to do it d',
 'is upset that he can t update his facebook by texting it and might cry as a result school today also blah ',
 ' kenichan i dived many times for the ball managed to save 50 the rest go out of bounds',
 'my whole body feels itchy and like its on fire',
 ' nationwideclass no it s not behaving at all i m mad why am i here because i can t see you all over there ']

In [ ]:
# save clearned txt

output_path = 'data/cleaned_data.txt'

os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(cleand_dataset))

In [ ]:
model_name = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained()